In [ ]:

valid_filenames = [] 
valid_vectors = []

all_files = os.listdir(image_dir)

for filename in all_files:
    path = os.path.join(image_dir, filename)
    
    try:
        # A. Ładowanie obrazu
        image = Image.open(path).convert("RGB")
        
        # B. Przetwarzanie (Processor)
        inputs = processor(images=image, return_tensors="pt")
        
        # C. Model + Normalizacja
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            # NORMALIZACJA (Kluczowe!)
            normalized_vec = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
            
        # D. ZAPIS ATOMOWY (Obie listy rosną RAZEM)
        valid_vectors.append(normalized_vec)
        valid_filenames.append(filename)
        
    except Exception as e:
        # Jeśli cokolwiek pójdzie nie tak, NIE dodajemy ani wektora, ani nazwy.
        # Dzięki temu listy zawsze są idealnie równe.
        # print(f"Pominięto {filename}: {e}") # Opcjonalne logowanie błędów
        pass

# 3. BUDOWA MACIERZY
if len(valid_vectors) > 0:
    db_features = torch.cat(valid_vectors, dim=0)
    print(f"✅ Zbudowano bazę: {len(valid_filenames)} zdjęć.")
else:
    print("❌ Błąd: Nie załadowano żadnych zdjęć.")
    exit()

# --- WYSZUKIWANIE ---

print(f"\n🔎 Szukam: '{target_query}'")
# Opcjonalnie: Tłumacz (odkomentuj jeśli wpisujesz po polsku)
# translator = GoogleTranslator(source='auto', target='en')
# target_query = translator.translate(target_query)

# 4. Przetwarzanie tekstu
text_input = processor(text=f"a photo of a {target_query}", return_tensors="pt", padding="max_length")
with torch.no_grad():
    text_emb = model.get_text_features(**text_input)
    text_emb = text_emb / text_emb.norm(p=2, dim=-1, keepdim=True)

# 5. Obliczenie wyników
scores = text_emb @ db_features.T
probs = torch.sigmoid((scores * model.logit_scale.exp()) + model.logit_bias)

# 6. Wyświetlenie TOP 3
top_vals, top_idxs = torch.topk(probs, k=3)
top_idxs = top_idxs[0].tolist()
top_vals = top_vals[0].tolist()

print("\n--- WYNIKI ---")
for i, idx in enumerate(top_idxs):
    filename = valid_filenames[idx] # Tu bierzemy nazwę z pewnej listy
    score = top_vals[i] * 100
    print(f"#{i+1}: {filename} (Pewność: {score:.1f}%)")